In [1]:
import torch
from torch import nn
from PIL import Image

In [2]:
def corr2d(x,k):
    h,w=k.shape
    result_h=x.shape[0]-h+1
    result_w=x.shape[1]-w+1
    result=torch.zeros((result_h,result_w))
    for i in range(result_h):
        for j in range(result_w):
            result[i,j]=(x[i:i+h,j:j+w]*k).sum()
    return result
class Conv2D(nn.Module):
    def __init__(self):
        super(Conv2D, self).__init__()
        self.bias=nn.Parameter(torch.zeros(1))
    def forward(self, x,k):
        r=corr2d(x,k)
        return r+self.bias

In [3]:
def show_image_from_tensor(x):
    if isinstance(x, torch.Tensor):
        x=x.numpy()*255
    Image.fromarray(x).show()

In [4]:
x=torch.ones([60,80])
x[:,20:60]=0
k=torch.tensor([[1,-1]])
Y=corr2d(x,k)

In [5]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
x=x.reshape(1,1,60,80)
Y=Y.reshape(1,1,60,79)

In [6]:
print(Y)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])


In [7]:
loss=nn.MSELoss()
for i in range(10000):
    # conv2d.zero_grad()
    loss.zero_grad()
    Y_hat=conv2d(x)
    l=loss(Y_hat,Y)
    l.backward()

    conv2d.weight.data[:]-=0.01*conv2d.weight.grad

In [8]:
conv2d.weight.data

tensor([[[[ 1.3048, -1.8903]]]])

### 卷积神经网络超参数

填充和步幅

In [21]:
def com_conv2d(conv2d,x):
    x=x.reshape((1,1)+x.shape)
    y=conv2d(x)
    return y

conv2d=nn.Conv2d(1,1,kernel_size=(3,3),padding=(1,1),bias=False)

In [22]:
x=torch.ones([6,6])

In [23]:
com_conv2d(conv2d,x).shape

torch.Size([1, 1, 6, 6])

通道数